## **Handling All Imports**

---

All necessary, non-nitrates packages and modules are imported first. Afterwards, nitrates is imported along with many specific function imports from nitrates's modules.

In [55]:
import numpy as np
from astropy.io import fits
from astropy.table import Table, vstack
from astropy.wcs import WCS
import os
from scipy import optimize, stats, interpolate
from scipy.integrate import quad
import argparse
import time
import multiprocessing as mp
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib import cm
import sys
import pandas as pd
pd.options.display.max_columns = 250
pd.options.display.max_rows = 250
import healpy as hp
from copy import copy, deepcopy
# sys.path.append('BatML/')
import logging, traceback
import sys
#logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)

In [56]:
#%matplotlib inline
import nitrates
from nitrates.config import rt_dir, solid_angle_dpi_fname
from nitrates.lib import get_conn, det2dpi, mask_detxy, get_info_tab, get_twinds_tab, ang_sep, theta_phi2imxy, \
    imxy2theta_phi, convert_imxy2radec, convert_radec2thetaphi, convert_radec2imxy
from nitrates.response import RayTraces
from nitrates.models import Cutoff_Plaw_Flux, Plaw_Flux, get_eflux_from_model, Source_Model_InOutFoV, \
    Bkg_Model_wFlatA, CompoundModel, Point_Source_Model_Binned_Rates, im_dist
from nitrates.llh_analysis import parse_bkg_csv, LLH_webins, NLLH_ScipyMinimize_Wjacob
print(nitrates.config.NITRATES_RESP_DIR, rt_dir)

/Volumes/WD/Development/Programming/NITRATES_RESP_DIR_PIP/ /Volumes/WD/Development/Programming/NITRATES_RESP_DIR_PIP/ray_traces_detapp_npy


## **Establishing Working Directories and Connecting to Results Database**
---

The path to the NITRATES_RESP_DIR directory gets stored in a os.environ object under the variable, "NIRATES_RESP_DIR". This will be user defined according to the user's own directory system.

In [57]:
os.environ['NITRATES_RESP_DIR'] = '/Volumes/WD/Development/Programming/NITRATES_RESP_DIR_PIP/'

The F646018360 directory is created within the NITRATES_RESP_DIR directory. Within this directory, the files contained within the tar file, csv_files.tar.gz, are unpacked.

In [58]:
os.system(f'cp -r ./F646018360 {nitrates.config.NITRATES_RESP_DIR}')
os.system(f'tar -xzf csv_files.tar.gz -C {os.path.join(nitrates.config.NITRATES_RESP_DIR,"F646018360/")}')

0

After setting up the NITRATES_RESP_DIR directory as described in the github repository, **NITRATES_path** stores the path to this directory found within config.py.

Within the NITRATES_RESP_DIR directory is the F646018360 directory. Its path is stored in **work_dir**.

In [59]:
NITRATES_path=nitrates.config.NITRATES_RESP_DIR
work_dir = os.path.join(NITRATES_path, 'F646018360')

The output of this notebook will be stored within a 'results.db' file within the F646018360 directory. In order to work within the file, a database connection must be opened to allow sqlite3 to work with it. The variable **conn**, stores the connection object which represents the connection to this database. 

In [60]:
conn = get_conn(os.path.join(work_dir,'results.db'))

Within the results.db database is the info tab. The info tab for this particular database stores the time start, time stop, and trigger time along with other values. The variable, **info_tab**, uses the get_info_tab function using the connection object, **conn**, to retrieve the info tab.

In [61]:
info_tab = get_info_tab(conn)

# To view the info_tab data, uncomment the following line
#info_tab

## **Setting Up Energy Bins**
---

The energy bins will be defined through two numpy arrays. **ebins0** will contain the lower bounds for each bin while **ebins1** will contain the upper bounds for each bin. 



**ebins0** is initially set up with energies of 15.0keV, 24.0keV, 35.0keV, 48.0keV, and 64.0keV. 

In [62]:
ebins0 = np.array([15.0, 24.0, 35.0, 48.0, 64.0])

This array is then appended by 5 additional points spaced out logarithmically between 84.0keV and 500.0keV. Since the detectors are only callibrated energies of up to 350keV, the last energy of 500.0keV is discarded

In [63]:
ebins0 = np.append(ebins0, np.logspace(np.log10(84.0), np.log10(500.0), 5+1))[:-1]

The entire array is rounded to one decimal place. The last energy is discarded again since 350.0 keV is our uppermost bound on registerable energies. This energy will be added at the end of **ebins1** instead.

In [64]:
ebins0 = np.round(ebins0, decimals=1)[:-1]

Now **ebins1** will be set up. As mentioned before, these will contain the upper bounds for each of the energy bins.

This array is constructed out of **ebins0** starting at its second element up to its final element plus the additional 350.0keV energy. The total number of energy bins is stored in **nebins**, equivalent to the length of **ebins0**



In [65]:
ebins1 = np.append(ebins0[1:], [350.0])
nebins = len(ebins0)
print("Nebins: ", nebins)
print("ebins0: ", ebins0)
print("ebins1: ", ebins1)

Nebins:  9
ebins0:  [ 15.   24.   35.   48.   64.   84.  120.  171.5 245. ]
ebins1:  [ 24.   35.   48.   64.   84.  120.  171.5 245.  350. ]


## **Trigger Time and Setting Up Time Window**
---

The trigger time is the exact timestamp that an event of interest occurs. In this scenario, the event of interest is a potential GRB event. This timestamp is given as the mission ellapsed time, the time ellapsed since mission start on January 1, 2001 00:00:00 UTC. 

Here, the trigger time is extracted from the **info_tab** table and is printed.

In [66]:
trigger_time = info_tab['trigtimeMET'][0]
print('trigger_time: ', trigger_time)

trigger_time:  646018383.1787


In order to capture the full evolution of the GRB, a window of time is constructed around **trigger_time**. The beginning of this window, **t_start**, occurs 1e3 seconds before **trigger_time** and the end of this window, **t_end**, occurs 1e3 seconds after **trigger_time**. 

In [67]:
t_end = trigger_time + 1e3
t_start = trigger_time - 1e3

## **Reading in Event Data and Designating Good Time Intervals**
---

The event data is stored in the filter_evdata.fits file within **work_dir**. The variable, **evfname**, stores the path to this file. Using this path, **ev_data** reads in the event data stored within the file.

In [68]:
evfname = os.path.join(work_dir,'filter_evdata.fits')
ev_data = fits.open(evfname)[1].data

# To view the ev_data data, uncomment the following line
#ev_data

The GTI or "Good Time Interval" are intervals of time for which the instruments are calibrated and the craft is not in motion. This data is stored in the filter_evdata.fits file.

**GTI_PNT** stores the good time intervals for which the craft is actively pointing at a fixed location in space. Similarly, **GTI_SLEW** stores the good time intervals for which the spacecraft is slewing to a new pointing orientation.

In [69]:
GTI_PNT = Table.read(evfname, hdu='GTI_POINTING')
GTI_SLEW = Table.read(evfname, hdu='GTI_SLEW')

## **Reading in Attitude Data**
---

The attitude.fits file stores data relating to where the instrument is pointed. **attfile** opens the file and stores the attitude data. 

In [70]:
attfile = fits.open(os.path.join(work_dir,'attitude.fits'))[1].data

# To view the attfile data, uncomment the following line
#attfile

From **attfile**, **att_ind** finds the appropriate table index corresponding to the trigger time. Finally, **att_quat** retrieves the attitude data in quaternionic form at index **att_ind**. This quaternion is printed.

In [71]:
att_ind = np.argmin(np.abs(attfile['TIME'] - trigger_time))
att_quat = attfile['QPARAM'][att_ind]
print('attitude quaternion: ', att_quat)

attitude quaternion:  [-0.03597053  0.2345147  -0.64420835  0.72712074]


## **Managing Defective Detectors and Masked Detectors**
---

Not all detectors within the detector array may be functioning. The detmask.fits file stores information on which pixels in the detector array are currently functioning and which ones aren't. **dmask** stores the data from the detmask.fits file.


In [72]:
dmask = fits.open(os.path.join(work_dir,'detmask.fits'))[0].data

# To view the dmask data, uncomment the following line
#dmask

Within **dmask**, detectors labelled 0 are considered "good" and detectors labelled 1 are "bad". The total number of good detectors is stored in **ndets** and is printed. 

In [73]:
ndets = np.sum(dmask==0)
print("Ndets: ", np.sum(dmask==0))

Ndets:  14932


The positions of the detectors within the detector array are indicated by their DETX and DETY coordinates. The function, mask_detxy, gets rid of event data for which the detectors are masked (i.e. dmask at DETX and DETY is 1). This data is stored into **mask_vals**.

In [74]:
mask_vals = mask_detxy(dmask, ev_data)

Finally, **bl_dmask** is the matrix of **dmask** after evaluating whether the entries of **dmask** are equal to 0.

In [75]:
bl_dmask = (dmask==0.)

# To view the bl_dmask data, uncomment the following line
#bl_dmask

## **Narrowing Search To Find GRB Event Data**

---

The event data can now be queried for data only relevant to the GRB event. From **ev_data**, **bl_ev** retrieves the indices of the event data for which (in order), the event flags are 0 indicating no errors occured during the event, the energy sits between 14keV and 500keV, the mask values are 0, and the event takes place within the timeframe indicated by **t_start** and **t_end**. 

In [76]:
bl_ev = (ev_data['EVENT_FLAGS']<1)&\
        (ev_data['ENERGY']<=500.)&(ev_data['ENERGY']>=14.)&\
        (mask_vals==0.)&(ev_data['TIME']<=t_end)&\
        (ev_data['TIME']>=t_start)

The number of corresponding events are printed and **ev_data0** stores the corresponding event data.

In [77]:
print("Nevents: ",np.sum(bl_ev))
ev_data0 = ev_data[bl_ev]

# To view the ev_data0 data, uncomment the following line
#ev_data0

Nevents:  1367885


---

## **Building Up a Simulated GRB Sourcee**
---

In this section, a GRB is generated at specific coordinates with a flux model as well as a ray trace object. 


We consider a GRB with known values for it's location in right ascension and declination. Those values are 233.117 and -26.213 stored by variables **ra** and **dec** respectively.

In [78]:
ra, dec = 233.117, -26.213 

The **ra** and **dec** values are converted into detector spherical coordinates (theta,phi) using the function, convert_radec2thetaphi. The function uses the **ra** and **dec** values as well as the quaternionic form of the attitude as arguments. These coordinates are then printed.

In [79]:
theta, phi = convert_radec2thetaphi(ra, dec, att_quat)
print(theta, phi)

38.54132137017975 137.65241966813443


Likewise, the **ra** and **dec** values can be converted into (imx,imy) coordinates in another instrument coordinate system using the convert_radec2imxy function. These coordinates are printed as well.

In [80]:
imx, imy = convert_radec2imxy(ra, dec, att_quat)
print(imx, imy)

-0.5887551341212707 -0.5366203642198198


A GRB can be described using a cutoff power law model, using the parameters contained by **flux_params**. The normalization is given by 'A', the photon index is given by 'gamma', and the cutoff value is given by the value 'Epeak'. In this example light source, these parameter values are 1.0, 0.5, and 1e2keV, respectively. The equation for the cutoff power law is given by:

$$ f(E) = A(\frac{E}{E_{piv}})^{\gamma} exp[-\frac{(2 - \gamma)E}{E_{peak}}]$$

where $$E_{piv} = 100\textrm{keV} $$
A separate flux model is generated in the form of a cutoff powerlaw using Cutoff_Plaw_Flux with an energy cutoff of E0=100.0keV. This flux model is stored by **flux_mod**.

In [81]:
flux_params = {'A':1.0, 'gamma':0.5, 'Epeak':1e2}
flux_mod = Cutoff_Plaw_Flux(E0=100.0)

## **Creating a RayTraces Object**
---

A RayTraces object is created using the path to the directory, ray_traces_detapp_npy, stored within the **rt_dir** variable in config.py. If **rt_dir** was not set up, the variable can be replaced by a string representing the path to the directory. The RayTraces object is given the variable name, **rt_obj**.

In [82]:
print(nitrates.config.NITRATES_RESP_DIR)
print(rt_dir)
rt_obj = RayTraces(rt_dir)

/Volumes/WD/Development/Programming/NITRATES_RESP_DIR_PIP/
/Volumes/WD/Development/Programming/NITRATES_RESP_DIR_PIP/ray_traces_detapp_npy


Here, the ray tracing process is simulated for a source at the prior defined **imx** and **imy** instrument coordinates. The result is an array of ray traces onto the detector plane of the illumination fractions, i.e. how much each detector "sees" this source. This array is stored into the variable, **rt**. Its shape is printed.

In [83]:
rt = rt_obj.get_intp_rt(imx, imy)
print(np.shape(rt))

# To view the rt data, uncomment the following line
#rt

(173, 286)


## **The Source_Model_InOutFoV Class**
---
In this section, the Source_Model_InOutFoV class is explored in detail. This class handles all of the forward modelling that is used for the likelihood analysis.

A Source_Model_InOutFoV object can be used to get Detector Plane Images. A Detector Plane Image shows the accumulation of photon counts in each detector across the detector array. For every combination of photon energy and sky location, there are response files that have been pre-calculated and stored across multiple directories. To find a particular response, it can be found according to the particular theta and phi values for the response. (MAKE SURE THIS IS!!) The responses are then assembled to create a Detector Response Matrix (DRM) which is used to calculate expected photon counts for each detector and energy bin. 

### .__init__()
---
 A Source_Model_InOutFoV object is constructed after taking in a flux model, the energy bins, the detector pixels, a **RayTraces** object, among other arguments. For the full list of arguments, see the __init__() definition below :


```
def __init__(self, flux_model,
                 ebins, bl_dmask, rt_obj,
                 name='Signal', use_deriv=False,
                 use_prior=False, resp_tab_dname=None,
                 hp_flor_resp_dname=None, comp_flor_resp_dname=None):
```


The flux_model gets saved into a class variable, **self.fmodel**, while the energy bins get stored in **self.ebins**. The lower bounds and upper bounds for the energy bins are specifically stored into **self.ebins0** and **self.ebins1**. The number of energy bins gets stored in **nebins**


```
self.ebins = ebins
self.ebins0 = ebins[0]
self.ebins1 = ebins[1]
nebins = len(self.ebins0)
```

The paths to the direct response directory (RESP_TAB_DNAME), florescence response directory (HP_FLOR_RESP_DNAME), and the compton-plus-florescence response directory (COMP_FLOR_RESP_DNAME) are stored in local class variables **.resp_dname**, **.flor_resp_dname**, and **.comp_flor_resp_dname**, respectively.

The class function .get_batxys() which will be described below, sets the values for the class variables **self.batxs** and **self.batys**.

For the purposes of error propogation, the errors for florescence, compton and florescence, non-florescence, coded, and non-coded responses are declared.

```
self.flor_err = 0.2
self.comp_flor_err = 0.16
self.non_flor_err = 0.12
self.non_coded_err = 0.1
self.coded_err = 0.05
```

The RayTraces object that was passed as an argument gets stored into a class variable, **self.rt_obj**.

The following variables are called upon when running **.set_theta_phi()** which will be discussed later. The values for **._resp_phi** and **._resp_theta** are set to NaN. The update value, **._resp_update**, is set up as 5.0 degrees. Similarly for the transmission files, the values for **._trans_phi** and **._trans_theta** are set to NaN while the **.trans_update** value is set to 5e-3 degrees. 

In [84]:
%%time
# will by default use the resp directories from config.py
sig_mod = Source_Model_InOutFoV(flux_mod, [ebins0,ebins1], bl_dmask,\
                                rt_obj, use_deriv=True)
# or the paths can be given
# resp_tab_dname = '/path/to/resp_tabs_ebins/'
# hp_flor_resp_dname = '/path/to/hp_flor_resps/'
# comp_flor_resp_dname = '/path/to/comp_flor_resps/'
# sig_mod = Source_Model_InOutFoV(flux_mod, [ebins0,ebins1], bl_dmask,\
#                                 rt_obj, use_deriv=True,\
#                                 resp_tab_dname=resp_tab_dname,\
#                                 hp_flor_resp_dname=hp_flor_resp_dname,\
#                                 comp_flor_resp_dname=comp_flor_resp_dname)

CPU times: user 789 µs, sys: 508 µs, total: 1.3 ms
Wall time: 14.7 ms


### .get_batxys()

---

The **.get_batxys()** function takes x and y indices from the detector coordinates given through **self.bl_dmask** and retrieves the corresponding physical coordinates within the BAT instrument coordinate system. The function makes use of the detxy2batxy(xinds, yinds) function to retrieve the coordinates. The resulting x and y coordinates are stored as class variables **.batxs** and **.batys**.

In [85]:
sig_mod.get_batxys()

# To view the .batxs and .batys, uncomment the following lines
print(sig_mod.batxs)
print(sig_mod.batys)

[-52.29 -51.45 -51.03 ...  59.01  59.43  59.85]
[-36.12 -36.12 -36.12 ...  36.12  36.12  36.12]


### .set_theta_phi(theta, phi)

---

The **.set_theta_phi(theta, phi)** function takes in arguments for theta and phi and updates the class variables, **.theta** and **.phi**, with the new values.

These values are compared to the values for class variables **.resp_theta** and **.resp_phi**. These variables hold the values for the theta and phi values that were last used to query the response files. If the angle separation between the two pairs of variables is larger than **._resp_update**, then new response files must be queried from using the new theta and phi values. The angle separation is calculated using the function **ang_sep()**, which returns the angle separation between  (**.theta**, **.phi**) and (**.resp_theta**, **.resp_phi**).

If the angle separation is less than **._resp_update**, we must also check if the angle separation is larger than **._trans_theta**. If so, then the transmission values, also queried by theta and phi, must be updated.

In [86]:
#The theta and phi values prior to running set_theta_phi()
sig_mod.set_theta_phi(theta, phi)

(0.0, 30.48, -14.117)
(0.0, 30.48, -14.117)


/Volumes/WD/Development/Programming/NITRATES_RESP_DIR_PIP/swift-bat/lib/python3.9/site-packages/nitrates/response/response.py:1499: RuntimeWarning: invalid value encountered in divide
  self.E_A0s = (self.orig_photonEs[self.Einds1] - self.photonEs) /\


(36.0, 45.0)
2.652419668134428
42.34758033186557
max rt: 0.8604
initing ResponseDPI, with fname
/Volumes/WD/Development/Programming/NITRATES_RESP_DIR_PIP/resp_tabs_ebins/drm_theta_36.0_phi_30.0_.fits
initing ResponseDPI, with fname
/Volumes/WD/Development/Programming/NITRATES_RESP_DIR_PIP/resp_tabs_ebins/drm_theta_36.0_phi_45.0_.fits
initing ResponseDPI, with fname
/Volumes/WD/Development/Programming/NITRATES_RESP_DIR_PIP/resp_tabs_ebins/drm_theta_45.0_phi_30.0_.fits
initing ResponseDPI, with fname
/Volumes/WD/Development/Programming/NITRATES_RESP_DIR_PIP/resp_tabs_ebins/drm_theta_45.0_phi_45.0_.fits


### .set_flux_params(flux_params)
---

The **.set_flux_params(flux_params)** function takes a list of parameters, flux_params, as an argument and updates the class variable, **.flux_param**, with the new values.

With the new parameters set, the following normed dpis are set: **.normed_photon_fluxes**, **.normed_comp_flor_rate_dpis**, **.normed_photoe_rate_dpis**, **.normed_rate_dpis**, and lastly,  **.normed_err_rate_dpis**. The normed dpis are useful for setting dpis while varying parameters such as the amplitude, A.

### .get_rate_dpis(params)
---

The **.get_rate_dpis(params)** function returns the detector plane images (or dpis). The function takes a list of parameters that contain values for theta, phi, and the normalization constant, A. 

For the new angles, if the angle separation is larger than the **_trans_update** value, then .set_theta_phi() and .set_flux_params() are run again for the new parameters. (not sure what imx and imy calculation do within this function) Finally, the rate_dpis are calculated after multiplying **.normed_rate_dpis** by the normalization constant, A, and returned.


In [87]:
sig_mod

Here, the theta and phi values are set using **set_theta_phi()** using the theta and phi values of the GRB source.

In [88]:
%%time
sig_mod.set_theta_phi(theta, phi)

CPU times: user 48 µs, sys: 55 µs, total: 103 µs
Wall time: 107 µs


## **Diffuse and Point Source Models**
---

The purpose of building a model is to eventually calculate the expected number of counts detected per detector and per energy bin as a function of our set of parameters. With these expected counts, they can be compared to the actual counts detected with the likelihood analysis. The models are split between the diffuse and point source models, which shall be described below.

### **Diffuse Model**
The diffuse model describes the expected distribution of counts $\lambda_{ij}$ for detector i and energy bin j:

$$\lambda_{ij}^{diff} = (\Omega_i\phi^{b}_{j} + r^{b}_{j})T$$

where $\Omega_i$ is the unblocked solid angle for detector i, $\phi^{b}_{j}$ is the rate per detector solid angle, and $r^{b}_{j}$ is the rate per detector. 

### **Point Source Model**
The point source model is somewhat different in that it is dependent on the source's positions ($\phi$, $\theta$). It likewise describes the expected distribution of counts for the ith detector and jth energy bin as:

$$\lambda_{ij}^{PS} = T\sum_{l}DRM_{ijl}(\theta,\phi)f(E_{\gamma} = E_{\gamma,l})\Delta E_{\gamma, l}$$

where T is the exposure, $DRM_{ijl}(\theta, \phi)$ is the detector response matrix for the particular source position of $(\theta, \phi)$, $f(E_{\gamma} = E_{\gamma,l})$ is the photon spectrum in energy bin $E_{\gamma, l}$, and (what is $\Delta E_{\gamma, l}$?)

For more information, refer to sections 4.1.1 and 4.1.2.

## **Background Model**
---

The background model encompasses any of the non-GRB sources that show up in the image. This can include contributions from the cosmic ray background or any number of sources that are known to be sources of cosmic rays. 

The background model consists mainly of a diffuse model from the contributions from the cosmic ray background combined with some point source models for each of the known sources. (sections 4.1.1 and 4.1.2) 

A collection of these sources is stored in the bkg_estimation.csv file. The path to this file is stored bkg_fname.

(need explanation of solid_ang_dpi)


In [89]:
bkg_fname = os.path.join(work_dir,'bkg_estimation.csv')
solid_ang_dpi = np.load(solid_angle_dpi_fname)

With bkg_estimation.csv read, the function parse_bkg_csv() parses through the  file in order to create point source models for the background sources. The function returns the following objects:

**bkg_df** -


**bkg_name** - Name for the background that will be used for background-related parameters.


**PSnames** - List of strings of the astronomical names of any sources of cosmic rays found within the background  

**bkg_mod** - The background model

**ps_mods** - List of corresponding point source models for each of the astronomical objects listed in **PSnames**

In [90]:
bkg_df, bkg_name, PSnames, bkg_mod, ps_mods = parse_bkg_csv(bkg_fname, solid_ang_dpi,\
                    ebins0, ebins1, bl_dmask, rt_dir)

['4U 1700-377', 'GRO J1655-40', 'GX 339-4', 'Sco X-1']


(need re-explanation on has_deriv and why it must be set to False)

The background model **bkg_mod** is added to a list of background models **bkg_mod_list**. If there are any point source models in **ps_mods** they are added to **bkg_mod_list** and has_deriv is set to False for each of them. With all the models compiled into a single list, the background model redefined as a compound model combining all of the point source models with the original **bkg_mod**.


In [91]:
bkg_mod.has_deriv = False
bkg_mod_list = [bkg_mod]
Nsrcs = len(ps_mods)
if Nsrcs > 0:
    bkg_mod_list += ps_mods
    for ps_mod in ps_mods:
        ps_mod.has_deriv = False
    bkg_mod = CompoundModel(bkg_mod_list)

The trigger time is used again to retrieve a dictionary of parameters at the trigger time index. **bkg_row** refers to this dictionary of parameters. The values of the paramameters of the background model are retrieved from **bkg_row** and stored as another dictionary, **bkg_params**. Below is a description of the output of calling **bkg_params**.

The first set of parameters have the form '_bkg_rate_0', '_bkg_rate_1', etc. These rates refer to contributions to the background from diffuse sources measured in counts per second. The number index in the parameter name refers to the corresponding energy bin for which the rate is measured (e.g the parameter '_bkg_rate_0' is the counts/s in energy bin 0 attributable to diffuse sources). 

Next are the parameters of the form '_flat_0', '_flat_1', etc. These parameters refer to the rate per detector parameters in the diffuse model, $r^{b}_{j}$. Their values are the fraction of the above background rate that is the result of the rate, $r^{b}_{j}$. (e.g '_flat_0' is the fraction of '_bkg_rate_0' attributable to the rate per detector $r^{b}_{0}$)

For each of the point sources within the background, the parameters corresponding to their imx and imy coordinates are listed along with the unmasked rates measured per energy bin.

In [92]:
tmid = trigger_time
bkg_row = bkg_df.iloc[np.argmin(np.abs(tmid - bkg_df['time']))]
bkg_params = {pname:bkg_row[pname] for pname in\
            bkg_mod.param_names}
bkg_name = bkg_mod.name
bkg_params

{'Background_bkg_rate_0': 0.0913703220701183,
 'Background_bkg_rate_1': 0.0661578002239374,
 'Background_bkg_rate_2': 0.0400898569026105,
 'Background_bkg_rate_3': 0.0394919934207801,
 'Background_bkg_rate_4': 0.0346780266835281,
 'Background_bkg_rate_5': 0.0351076584048365,
 'Background_bkg_rate_6': 0.033602492003955,
 'Background_bkg_rate_7': 0.0248610826847874,
 'Background_bkg_rate_8': 0.0176399476145076,
 'Background_flat_0': 0.0,
 'Background_flat_1': 0.0,
 'Background_flat_2': 0.1859632649428977,
 'Background_flat_3': 0.0766344244533236,
 'Background_flat_4': 0.3020855890115875,
 'Background_flat_5': 0.7611510180823338,
 'Background_flat_6': 0.8091430254293075,
 'Background_flat_7': 1.0,
 'Background_flat_8': 1.0,
 '4U 1700-377_imx': -0.0981485305770971,
 '4U 1700-377_imy': -0.4742076074486664,
 '4U 1700-377_rate_0': 0.0124901359218975,
 '4U 1700-377_rate_1': 0.0072425264886238,
 '4U 1700-377_rate_2': 0.0045295645864628,
 '4U 1700-377_rate_3': 0.0016876913514016,
 '4U 1700-377_r

In [94]:
pars_ = {}
pars_['Signal_theta'] = theta
pars_['Signal_phi'] = phi
for pname,val in list(bkg_params.items()):
    pars_[bkg_name+'_'+pname] = val
for pname,val in list(flux_params.items()):
    pars_['Signal_'+pname] = val
pars_

# creating a dictionary for the data in the output of the previous cell and changing name so everything is prepended
# and has signal_A, gamma, Epeak.

{'Signal_theta': 38.54132137017975,
 'Signal_phi': 137.65241966813443,
 'Background+4U 1700-377+GRO J1655-40+GX 339-4+Sco X-1_Background_bkg_rate_0': 0.0913703220701183,
 'Background+4U 1700-377+GRO J1655-40+GX 339-4+Sco X-1_Background_bkg_rate_1': 0.0661578002239374,
 'Background+4U 1700-377+GRO J1655-40+GX 339-4+Sco X-1_Background_bkg_rate_2': 0.0400898569026105,
 'Background+4U 1700-377+GRO J1655-40+GX 339-4+Sco X-1_Background_bkg_rate_3': 0.0394919934207801,
 'Background+4U 1700-377+GRO J1655-40+GX 339-4+Sco X-1_Background_bkg_rate_4': 0.0346780266835281,
 'Background+4U 1700-377+GRO J1655-40+GX 339-4+Sco X-1_Background_bkg_rate_5': 0.0351076584048365,
 'Background+4U 1700-377+GRO J1655-40+GX 339-4+Sco X-1_Background_bkg_rate_6': 0.033602492003955,
 'Background+4U 1700-377+GRO J1655-40+GX 339-4+Sco X-1_Background_bkg_rate_7': 0.0248610826847874,
 'Background+4U 1700-377+GRO J1655-40+GX 339-4+Sco X-1_Background_bkg_rate_8': 0.0176399476145076,
 'Background+4U 1700-377+GRO J1655-40+G

With the background and signal models assembled, a singular compound model can be created which combines the two models into one. **comp_mod** is a CompoundModel class object created using the **bkg_mod** and **sig_mod** variables.

In [95]:
comp_mod = CompoundModel([bkg_mod, sig_mod])

## **Calculating and Minimizing Log Likelihood**
---

The compound model can now be used to calculate the likelihood statistic. (display likelihood statistic equation). The parameters that minimize this statistic is pressumed to be the parameters that most likely corresponds to the observed data (bad phrasing).

The calculation of the log likelihood is done through the LLH_webins class object, **sig_llh_obj**, which utilizes the event data (**ev_data0**), the lower and upper bounds of the energy bins (**ebins0** and **ebins1**), and the detector data (**bl_dmask**). **sig_llh_obj** makes use of the compound model by invoking the .set_model() function with **comp_mod** as an argument. When initializing **sig_llh_obj**, the argument has_err is set to True in order to utilize the error values set up in the compound model.

In [96]:
sig_llh_obj = LLH_webins(ev_data0, ebins0, ebins1, bl_dmask, has_err=True)
sig_llh_obj.set_model(comp_mod)

The log likelihood minimization is handled by the NLLH_ScipyMinimize_Wjacob object, **sig_miner**, which is a wrapper of the scipy minimizer object. Using the .set_llh() function, it can act on **sig_llh_obj** to recalculate the log likelihood through variations of the parameters. 

In [97]:
sig_miner = NLLH_ScipyMinimize_Wjacob('')
sig_miner.set_llh(sig_llh_obj)

In order to cut down on the computation time, only one parameter vary during this minimization process. The 'Signal_A' parameter will be varied while the other parameters are held fixed. The rest of the parameters will be held constant. The parameters to be kept constant are listed by **fixed_pnames** with corresponding values **fixed_vals**. A transformation on each of the parameter variables can be supplied in a list **trans**. In this scenario, all of the fixed parameteres are supplied without any transformations. 

The minimizer object saves these fixed parameters and transformations through the functions .set_trans() and .set_fixed_params(). Since 'Signal_A' is to be varied, .set_fixed_params() is used with the option fixed=False to inform the minimizer that this parameter will be varied over.


In [98]:
fixed_pnames = list(pars_.keys())
fixed_vals = list(pars_.values())
trans = [None for i in range(len(fixed_pnames))]
sig_miner.set_trans(fixed_pnames, trans)
sig_miner.set_fixed_params(fixed_pnames, values=fixed_vals)
sig_miner.set_fixed_params(['Signal_A'], fixed=False)

The source model flux parameters are known *a priori* with the gamma and Epeak values set at 0.8 and 350.0, respectively.

In [99]:
#%%time
# setting gamma and Epeak for a "known" source
flux_params['gamma'] = 0.8
flux_params['Epeak'] = 350.0
sig_mod.set_flux_params(flux_params)

Likewise, the exact time window of the signal is known *a priori* between the initial time **t0** and **t1**. The signal likelihood object can set the time and duration of the LLH analysis using .set_time().

In [100]:
# Setting initial time and final time of known GRB event
t0 = trigger_time - 0.512
t1 = t0 + 2.048
sig_llh_obj.set_time(t0, t1)

The minimization process is run using the NLLH_ScipyMinimize_Wjacob function .minimize() which has three return objects. The first, **pars**, contains the set of parameters which minimize the likelihood. The second parameter, **nllh**, is the negative log likelihood which quantifies how good the fit is for the given model. The last, **res**, is a return object of the original Scipy minimizer object. 

In [101]:
#%%time
pars, nllh, res = sig_miner.minimize()

print(res)
print(nllh)
print(pars)

[      fun: 45894.07068100051
 hess_inv: <1x1 LbfgsInvHessProduct with dtype=float64>
      jac: array([0.00239975])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 8
      nit: 5
     njev: 8
   status: 0
  success: True
        x: array([0.006875])]
[45894.07068100051]
[[0.006875003655224625]]


The log likelihood is calculated for the background-only model so that it can be compared to the compound model. Setting the parameter 'Signal_A' to 1e-10 effectively cancels out the contribution from the signal model. Since we are not varying any of the parameters, there the log likelihood only needs to be calculated once using the .get_logprob() function.



In [102]:
%%time
# Evaluating log likelihood for background only model. This is accomplished by setting Signal_A to effectively zero. 
pars_['Signal_A'] = 1e-10
bkg_nllh = -sig_llh_obj.get_logprob(pars_)
print (bkg_nllh)
print (sqrtTS)

46038.74332720728
17.01015262757948
CPU times: user 3.43 ms, sys: 1.41 ms, total: 4.84 ms
Wall time: 4.07 ms


The likelihood ratio test statistic given by:

$$\lambda  = -2[LLH(\Theta_{bkg}) - LLH(\Theta_{sig}, \Theta_{bkg})]$$
 
 (citate the paper)

is used to compare the compare the background+signal model to the background model. This statistic is used to quantify the signicance of the signal+background model. In the following block, the likelihood ratio test statistic is calculated using the negative log likelihood found for the background model and the minimized background+signal model. 

In [103]:
sqrtTS = np.sqrt(2.*(bkg_nllh - nllh[0]))
print (sqrtTS)

17.01015262757948


(is this last import important at all?)

In [104]:
import nitrates as nt